In [335]:
import numpy as np
import pandas as pd
import pyarrow
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig

In [336]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print("Using device: " + device)

Using device: cpu


In [337]:
import urllib
import os
import json

# Corpus and dictionary files to use
corpus_file = 'democracy_reports_corpus.csv'
dictionary_file = 'dimension_dictionary.json'
corpus_file_url = "https://github.com/backdem/democracy-datasets/raw/main/democracy_reports_corpus.csv"
dictionary_file_url = "https://raw.githubusercontent.com/backdem/democracy-datasets/main/dimension_dictionary.json"

# Download datsets if not already downloaded
if not os.path.exists(corpus_file):
    urllib.request.urlretrieve(corpus_file_url, corpus_file)
if not os.path.exists(dictionary_file):
    urllib.request.urlretrieve(dictionary_file_url, dictionary_file)

def load_json_dict(dict_file):
    with open(dict_file, 'r') as file:
        dictionary = json.load(file)
        dictionary.append({
            'name': 'no_dimension',
            'words': []
        })
        return dictionary

dimension_dictionary = load_json_dict(dictionary_file)
print(f'Found topics: {[dim["name"] for dim in dimension_dictionary]}')
# Naive approach to label sentences with dictionary, producing 
# a masked label vector of the form [0, 0, 0, 0, 0, 1] where the 
# indicies match topics ['electoral', 'participatory', 'media', 'liberal_institution', 'liberal_rights', 'no_dimension']
def generate_label_vector(sentence, dict=dimension_dictionary):
    topics = [dim['name'] for dim in dict]
    matched_dim = 'no_dimension'
    for dim in dict:
        if matched_dim != 'no_dimension':
            break
        for w in dim['words']:
            if w in sentence.lower():
                matched_dim = dim['name']
                break
    return [int(t == matched_dim) for t in topics]
        

Found topics: ['electoral', 'participatory', 'media', 'liberal_institution', 'liberal_rights', 'no_dimension']


In [338]:
# Read csv file into Dataframe
df = pd.read_csv(corpus_file, dtype={'year': str, 'sentence': str}, comment='#')
# Print first row
print(df.sample(5))

                                                 sentence  \
218878  greco recommended to ensure that the relevant ...   
62974   However, the authorities regularly use violenc...   
129469  In addition, existing institutions are undereq...   
355742  such advertisements must contain no informatio...   
317181  greco takes note of the information provided a...   

                                                  section             country  \
218878                                               none            slovenia   
62974   ['Executive Summary', 'Score changes in 2015',...          azerbaijan   
129469                                                NaN  bosnia-herzegovina   
355742                                               none              france   
317181                                               none              poland   

        year                        source  
218878  2009                         greco  
62974   2015  freedomhouse_nations-transit  
129469  200

In [339]:
# Create label vector column
df_sentences_labels = pd.DataFrame(df['sentence'])
df_sentences_labels['label'] = df_sentences_labels['sentence'].apply(generate_label_vector)

In [340]:
from transformers import BertTokenizer
FRACTION_OF_DS_TO_USE = 0.1
MAX_LEN = 256
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 1
LEARNING_RATE = 1e-05
MAX_LOSS = 0.02
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)

In [341]:
# Use part of the dataset
new_df = df_sentences_labels.sample(frac=FRACTION_OF_DS_TO_USE, random_state=200)
# Display a sample of the dataset
new_df.sample(10)[['sentence', 'label']]

,sentence,label
15233,"Alleging electoral fraud, the opposition rejec...","[1, 0, 0, 0, 0, 0]"
207408,"During the period under review, the system of ...","[0, 0, 0, 0, 0, 1]"
372364,"by way of example, a former head of the centra...","[0, 1, 0, 0, 0, 0]"
194777,"In 1919, after Word War I, a mandate for the r...","[1, 0, 0, 0, 0, 0]"
6319,The Association of Free Trade Unions of Sloven...,"[0, 1, 0, 0, 0, 0]"
327667,"by contrast, no classes on these issues have s...","[0, 0, 0, 1, 0, 0]"
157529,The functioning and management of public educa...,"[0, 0, 0, 0, 0, 1]"
40138,Although not holding a formal government posit...,"[0, 0, 0, 0, 0, 1]"
180149,"Please cite as follows: Bertelsmann Stiftung, ...","[0, 0, 0, 0, 0, 1]"
431295,to unambiguously cover bribery of foreign arbi...,"[0, 0, 0, 0, 0, 1]"


In [342]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.comment_text = dataframe.sentence
        self.targets = self.data.label
        self.max_len = max_len

    def __len__(self):
        return len(self.comment_text)

    def __getitem__(self, index):
        comment_text = str(self.comment_text[index])
        comment_text = " ".join(comment_text.split())

        inputs = self.tokenizer.encode_plus(
            comment_text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [343]:
train_size = 0.8
train_dataset=new_df.sample(frac=train_size,random_state=200)
test_dataset=new_df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(new_df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (46073, 2)
TRAIN Dataset: (36858, 2)
TEST Dataset: (9215, 2)


In [344]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [345]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased')
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 6)
    
    def forward(self, ids, mask, token_type_ids):
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output

model = BERTClass()
model.to(device)

BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [346]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [347]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [348]:
def train(epoch):
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%500==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        if loss.item() < MAX_LOSS:
            break
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return loss

In [349]:
# Train
for epoch in range(EPOCHS):
    loss = train(epoch)
    print(f'Epoch {epoch} done')
    if loss.item() < MAX_LOSS:
        break

Epoch: 0, Loss:  0.6917052268981934
Epoch: 0, Loss:  0.13105759024620056
Epoch: 0, Loss:  0.10825284570455551
Epoch 0 done


In [350]:
def validation(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [351]:
for epoch in range(EPOCHS):
    outputs, targets = validation(epoch)
    outputs = np.array(outputs) >= 0.5
    accuracy = metrics.accuracy_score(targets, outputs)
    f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
    f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
    print(f"Accuracy Score = {accuracy}")
    print(f"F1 Score (Micro) = {f1_score_micro}")
    print(f"F1 Score (Macro) = {f1_score_macro}")

Accuracy Score = 0.9306565382528487
F1 Score (Micro) = 0.9369202226345084
F1 Score (Macro) = 0.8762412181303967


In [387]:
# Test our inputs
# Choose a sentence
sample_row = new_df.sample()[['sentence', 'label']]
sample_text = sample_row['sentence'].values[0]

# Encode the sentence
encoding = tokenizer.encode_plus(
  sample_text.lower(),
  add_special_tokens=True,
  max_length=MAX_LEN,
  padding='max_length',
  truncation=True,
  return_token_type_ids=True,
  return_tensors='pt'
)

input_ids = encoding["input_ids"]
mask = encoding["attention_mask"]
token_type_ids = encoding["token_type_ids"]
input_ids = input_ids.to(device, dtype=torch.long)
mask = mask.to(device, dtype=torch.long)
token_type_ids = token_type_ids.to(device, dtype=torch.long)

# Inference
output = model(input_ids, mask, token_type_ids)
predictions = output[0].tolist()
print(predictions)

[2.804351568222046, -2.9989209175109863, -3.4676501750946045, -3.5935776233673096, -3.6320676803588867, -2.935041904449463]


In [388]:
def get_dimension_from_prediction(v, dict=dimension_dictionary):
    dims = [dim["name"] for dim in dimension_dictionary]
    max = 0
    index = -1
    for i in range(len(v)):
        if v[i] > max:
            max = v[i]
            index = i
    return dims[index]


In [389]:
result_dim = get_dimension_from_prediction(predictions)
print(f'Statment "{sample_text}" was classified as: {result_dim}.')

Statment "all statements were, however, submitted in the 2009 elections to the european parliament." was classified as: electoral.
